In [1]:
import torch
import random
import math
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import networkx as nx
import sys


In [2]:
class DVRPSR_Dataset(Dataset):

    customer_feature = 4 # customer features location (x_i,y_i) and duration of service(d), appearance (u)

    @classmethod
    def create_data(cls,
                    batch_size = 2,
                    vehicle_count = 2,
                    vehicle_speed = 20, # km/hr
                    Lambda = 0.025, # request rate per min
                    dod = 0.5,
                    horizon = 400,
                    fDmean = 10,
                    fDstd = 2.5):


        # static customer counts V = Lambda*horizon*(1-dod)/(dod+0.5)
        V_static = int(Lambda*horizon*(1-dod)/(dod)+0.5)

        # total customer count
        V = int(Lambda*horizon/(dod) + 0.5)

        size = (batch_size, V, 1)
        
        # initialize the graph of vienna network
        graph = cls.initialize_graph()

        # get the coordinates of customers
        data_vienna = pd.read_csv('vienna_cordinates.csv')

        # get depot coordinates: Id, xcoords, ycoords
        depot = cls.get_depot_location(data_vienna)

        # get location of customers: id, xcoords, ycoords
        locations = cls.get_customers_coordinates(data_vienna, batch_size, V, depot)

        # get edges index and attributes, which is distance between one node to others n_i*n_j
        edges_index, edges_attributes = cls.get_edges_attributes(batch_size, graph, depot, locations, V)
        
        ### generate Static_Dynamic customer requests
        dynamic_request = cls.generateRandomDynamicRequests(batch_size,
                                                            V,
                                                            V_static,
                                                            fDmean,
                                                            fDstd,
                                                            Lambda,
                                                            horizon)

        customers = torch.zeros((batch_size,V,cls.customer_feature))
        customers[:,:,:2] = locations[:,:,1:]
        customers[:,:,2:4] = dynamic_request



        depo = torch.zeros((batch_size, 1, cls.customer_feature))
        depo[:,:,0:2] = torch.from_numpy(depot[0][1:])
        depo[:,:,2] =  0

        nodes = torch.cat((depo, customers), 1)
        
        dataset = cls(vehicle_count, vehicle_speed, horizon, nodes, V, 
                      edges_index, edges_attributes, customer_mask = None)
        
        return dataset
    
    def __init__(self, vehicle_count, vehicle_speed, horizon, nodes, V,
                 edges_index, edges_attributes, customer_mask=None):
        
        self.vehicle_count = vehicle_count
        self.vehicle_speed = vehicle_speed
        self.nodes = nodes
        self.vehicle_time_budget = horizon
        self.edges_index = edges_index
        self.edges_attributes = edges_attributes

        self.batch_size, self.nodes_count, d = self.nodes.size()

        if d!= self.customer_feature:
            raise ValueError("Expected {} customer features per nodes, got {}".format(
                self.customer_feature, d))

        self.customer_mask = customer_mask
        self.customer_count = V
        
        
    def initialize_graph():
    
        coordinates = pd.read_csv("vienna_dist.csv", header = None, sep=' ')
        coordinates.columns = ['coord1','coord2','dist']
        graph = nx.DiGraph()

        # add the rows to the graph for shortest path and distance calculations
        for _, row in coordinates.iterrows():
            graph.add_edge(row['coord1'], row['coord2'], weight=row['dist'])

        return graph


    def precompute_shortest_path(graph, start_node, end_node):

        shortest_path = nx.shortest_path(graph, start_node, end_node)

        # TODO: distance need to be normalized afterwords
        shortest_path_length = sum(graph.get_edge_data(u, v)['weight'] 
                                   for u, v in zip(shortest_path, shortest_path[1:]))

        return shortest_path, shortest_path_length 
    
    
    def get_distanceLL(lat1, lon1, lat2, lon2):
    
        R = 6371  # Radius of the Earth in kilometers

        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)

        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad

        a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        return distance
    

    def get_NearestNodeLL(lat, lon, lats, lons):
        nearest = (-1, sys.float_info.max)
        for i in range(len(lats)):
            dist = DVRPSR_Dataset.get_distanceLL(lat, lon, lats[i], lons[i])
            if dist < nearest[1]:
                nearest = (i, dist)
        return nearest[0]
    


    def get_depot_location(data_vienna):

        ll = (48.178808, 16.438460)
        lat = ll[0] / 180 * math.pi
        lon = ll[1] / 180 * math.pi
        lats = data_vienna['lats']
        lons = data_vienna['lons']
        depot = DVRPSR_Dataset.get_NearestNodeLL(lat, lon, lats, lons)
        depot_coordinates = np.array(data_vienna[data_vienna['id']==depot][['id','xcoords', 'ycoords']])

        return depot_coordinates
    
    def get_customers_coordinates(data_vienna, batch_size, customers_count, depot):
        
        torch.manual_seed(42)

        # Excluding depot id from the customers selection
        data_vienna_without_depot = data_vienna[data_vienna['id'] != int(depot[0][0])].reset_index()

        # Sample customers indices for all batches at once
        sampled_customers = torch.multinomial(torch.tensor(data_vienna_without_depot['id'], dtype=torch.float),
                                              num_samples=batch_size * customers_count, replacement=True)
        
        sampled_customers = sampled_customers.reshape(batch_size, customers_count)

        # Gather the sampled locations using the indices
        sampled_locations = data_vienna_without_depot.loc[sampled_customers.flatten()].reset_index(drop=True)

        # Reshape the locations to match the batch size
        locations = sampled_locations.groupby(sampled_locations.index // customers_count)

        # Create PyTorch tensors for the batched data
        locations_tensors = []
        for _, batch in locations:
            id_tensor = torch.tensor(batch['id'].values, dtype=torch.long)
            coords_tensor = torch.tensor(batch[['xcoords', 'ycoords']].values, dtype=torch.float)
            batch_tensor = torch.cat((id_tensor.unsqueeze(1), coords_tensor), dim=1)
            locations_tensors.append(batch_tensor)

        return torch.stack(locations_tensors)
    
    def c_dist(x1,x2):
        return ((x1[0]-x2[0])**2+(x1[1]-x2[1])**2)**0.5
    
    def get_edges_attributes(batch_size, graph, depot, locations, V):
    
        # all customers ID inclusing depot
        
        print('Initialzing edges')
        edge_depot = torch.zeros((batch_size, 1, 2))
        edge_depot[:,:,0] = depot[0][1]
        edge_depot[:,:,1] = depot[0][2]
        edge_data = torch.cat((edge_depot, locations[:,:,1:3]), dim=1)

        # generate edge index
        edges_index = []

        for i in range(V+1):
            for j in range(V+1):
                edges_index.append([i, j])
        edges_index = torch.LongTensor(edges_index)
        edges_index = edges_index.transpose(dim0=0,dim1=1)

        # generate nodes attributes
        edges_batch = []

        for batch in edge_data:
            edges = np.zeros((V+1, V+1, 1))
            for i, node1 in enumerate(batch):
                for j, node2 in enumerate(batch):
                    distance = DVRPSR_Dataset.c_dist(node1, node2)
                    edges[i][j][0] = distance

            edges = edges.reshape(-1, 1)
            edges_batch.append(torch.from_numpy(edges))

        return edges_index, torch.stack(edges_batch)
    
    
    def generateRandomDynamicRequests(batch_size=2 ,
                                      V=20,
                                      V_static=10,
                                      fDmean=10,
                                      fDstd=2.5,
                                      Lambda=0.025,
                                      horizon=400,
                                      dep = 0,
                                      u = 0):
        gen = random.Random()
        gen.seed() # uses the default system seed
        unifDist = gen.random # uniform distribution
        durDist = lambda: max(0.01, gen.gauss(fDmean, fDstd)) # normal distribution with fDmean and fDstd

        # TODO: in actual data , we need to add a depo node with corrdinate, which should be removed from selected
        #       nodes as well.

        requests = []
        for b in range(batch_size):

            static_request = []
            dynamic_request = []
            u = 0

            while True:
                unif = unifDist()
                u += -(1/Lambda) * math.log(unif)
                if u > horizon or len(dynamic_request) > (V-V_static+2):
                    break
                d = round(durDist(),2)
                while d<=0:
                    d = round(durDist(),2)

                dynamic_request.append([d, round(u,2)])

            for j in range(V-len(dynamic_request)):
                d = round(durDist(),2)
                while d<=0:
                    d = round(durDist(),2)
                static_request.append([d,0])

            request = static_request+dynamic_request
            random.shuffle(request)
            requests.append(request)

        return torch.tensor(requests)
    
    

    
    def __len__(self):
        return self.batch_size


    def __getitem__(self, i):
        if self.customer_mask is None:
            return self.nodes[i]
        else:
            return self.nodes[i], self.customer_mask[i]

    def nodes_generate(self):
        if self.customer_mask is None:
            yield from self.nodes
        else:
            yield from (n[m^1] for n,m in zip(self.nodes, self.customer_mask))  
            
            
    def normalize(self):
        loc_max, loc_min = self.nodes[:,:,:2].max().item(), self.nodes[:,:,:2].min().item()
        loc_max -= loc_min
        edge_max_length = self.edges_attributes[:,:,0].max().item()

        self.nodes[:,:,:2] -= loc_min
        self.nodes[:,:,:2] /= loc_max
        self.nodes[:,:,2:] /=self.vehicle_time_budget

        self.vehicle_speed *= self.vehicle_time_budget/edge_max_length
        self.vehicle_time_budget = 1
        self.edges_attributes /= edge_max_length
        return loc_max, 1

    def save(self, folder_path):
        torch.save({
            'veh_count':self.vehicle_count,
            'veh_speed':self.vehicle_speed,
            'nodes':self.nodes,
            'edges_index':self.edges_index,
            'edges_attributes':self.edges_attributes,
            'customer_count':self.customer_count,
            'customer_mask':self.customer_mask
        }, folder_path)

    @classmethod
    def load(cls, folder_path):
        return cls(**torch.load(folder_path))
        
        
        
        

In [3]:
data = DVRPSR_Dataset.create_data(batch_size=4, vehicle_count=2,vehicle_speed=1/3, Lambda=0.2, dod=0.75, horizon=600)

Initialzing edges


In [4]:
data.edges_index

tensor([[  0,   0,   0,  ..., 160, 160, 160],
        [  0,   1,   2,  ..., 158, 159, 160]])

## Environment

In [5]:
import torch


class DVRPSR_Environment:
    vehicle_feature = 8  # vehicle coordinates(x_i,y_i), veh_time_time_budget, total_travel_time, last_customer,
                         # next(destination) customer, last rewards, next rewards
    customer_feature = 4

    # TODO: change pending cost for rewards

    def __init__(self, data, nodes=None, customer_mask=None, 
                 pending_cost=1, 
                 dynamic_reward=0.2,
                 budget_penalty = 10):

        self.vehicle_count = data.vehicle_count
        self.vehicle_speed = data.vehicle_speed
        self.vehicle_time_budget = data.vehicle_time_budget

        self.nodes = data.nodes if nodes is None else nodes
        self.edge_index = data.edges_index
        self.edge_attributes = data.edges_attributes
        self.init_customer_mask = data.customer_mask if customer_mask is None else customer_mask

        self.minibatch, self.nodes_count, _ = self.nodes.size()
        self.distance_matrix = data.edges_attributes.view((self.minibatch, self.nodes_count, self.nodes_count))
        self.pending_cost = pending_cost
        self.dynamic_reward = dynamic_reward
        self.budget_penalty = budget_penalty

    def _update_current_vehicles(self, dest, customer_index, tau=0):

        # calculate travel time
        # TODO: 1) in real world setting we need to calculate the distance of arc
        # If nodes i and j are directly connected by a road segment (i, j) ∈ A, then t(i,j)=t_ij;
        # otherwise, t(i,j)=t_ik1 +t_k1k2 +...+t_knj, where k1,...,kn ∈ V are the nodes along the
        # shortest path from node i to node j.
        #      2) calculate stating time for each vehicle $\tau $, currently is set to zero
        
        # update vehicle previous and next customer id
        self.current_vehicle[:, :, 4] = self.current_vehicle[:, :, 5]
        self.current_vehicle[:, :, 5] = customer_index
        
        # get the distance from current vehicle to its next destination
        dist = torch.zeros((self.minibatch, 1))
        for i in range(self.minibatch):
            dist[i, 0] = self.distance_matrix[i][int(self.current_vehicle[i, :, 4])][int(self.current_vehicle[i, :, 5])]
            
        
        # total travel time    
        tt = dist / self.vehicle_speed

        # customers which are dynamicaly appeared
        dyn_cust = (dest[:, :, 3] > 0).float()

        # budget left while travelling to destination nodes
        budget = tau + tt + dest[:, :, 2]
        # print(budget, tau, tt, dest[:,:,2])

        # update vehicle features based on destination nodes
        self.current_vehicle[:, :, :2] = dest[:, :, :2]
        self.current_vehicle[:, :, 2] -= budget
        self.current_vehicle[:, :, 3] += tt
        self.current_vehicle[:, :, 6] = self.current_vehicle[:, :, 7]
        self.current_vehicle[:, :, 7] = -dist

        # update vehicles states
        self.vehicles = self.vehicles.scatter(1,
                                              self.current_vehicle_index[:, :, None].expand(-1, -1, self.vehicle_feature),
                                              self.current_vehicle)

        return dist, dyn_cust

    def _done(self, customer_index):
        
        self.vehicle_done.scatter_(1, self.current_vehicle_index, torch.logical_or((customer_index == 0), 
                                                                     (self.current_vehicle[:, :, 2] <= 0)))
        # print(self.veh_done, cust_idx==0,self.cur_veh[:,:,2]<=0, (cust_idx==0) | (self.cur_veh[:,:,2]<=0))
        self.done = bool(self.vehicle_done.all())

    def _update_mask(self, customer_index):

        self.new_customer = False
        self.served.scatter_(1, customer_index, customer_index > 0)

        # cost for a vehicle to go to customer and back to deport considering service duration
        cost = torch.zeros((self.minibatch, self.nodes_count,1))
        for i in range(self.minibatch):
            for j in range(self.nodes_count):
                dist_vehicle_customer_depot = self.distance_matrix[i][int(self.current_vehicle[i, :, 4])][j] + \
                                              self.distance_matrix[i][j][0]
                cost[i,j] = dist_vehicle_customer_depot
                
        cost = cost / self.vehicle_speed

        cost += self.nodes[:, :, None, 2]

        overtime_mask = self.current_vehicle[:, :, None, 2] - cost
        overtime_mask = overtime_mask.squeeze(2).unsqueeze(1)
        overtime = torch.zeros_like(self.mask).scatter_(1,
                                                        self.current_vehicle_index[:, :, None].expand(-1, -1, self.nodes_count),
                                                        overtime_mask < 0)

        self.mask = self.mask | self.served[:, None, :] | overtime | self.vehicle_done[:, :, None]
        self.mask[:, :, 0] = 0  # depot

    # updating current vehicle to find the next available vehicle
    def _update_next_vehicle(self):
        
        avail = self.vehicles[:, :, 3].clone()
        avail[self.vehicle_done] = float('inf')

        self.current_vehicle_index = avail.argmin(1, keepdim=True)
        self.current_vehicle = self.vehicles.gather(1, self.current_vehicle_index[:, :, None].expand(-1, -1, self.vehicle_feature))
        self.current_vehicle_mask = self.mask.gather(1, self.current_vehicle_index[:, :, None].expand(-1, -1, self.nodes_count))

    def _update_dynamic_customers(self):

        time = self.current_vehicle[:, :, 3].clone()

        if self.init_customer_mask is None:
            reveal_dyn_reqs = torch.logical_and((self.customer_mask), (self.nodes[:, :, 3] <= time))
        else:
            reveal_dyn_reqs = torch.logical_and((self.customer_mask ^ self.init_customer_mask), (self.nodes[:, :, 3] <= time))

        if reveal_dyn_reqs.any():
            self.new_customer = True
            self.customer_mask = self.customer_mask ^ reveal_dyn_reqs
            self.mask = self.mask ^ reveal_dyn_reqs[:, None, :].expand(-1, self.vehicle_count, -1)
            self.vehicle_done = torch.logical_and(self.vehicle_done, (reveal_dyn_reqs.any(1) ^ True).unsqueeze(1))
            self.vehicles[:, :, 3] = torch.max(self.vehicles[:, :, 3], time)
            self._update_next_vehicle()

    def reset(self):
        # reset vehicle (minibatch*veh_count*veh_feature)
        self.vehicles = self.nodes.new_zeros((self.minibatch, self.vehicle_count, self.vehicle_feature))
        self.vehicles[:, :, :2] = self.nodes[:, :1, :2]
        self.vehicles[:, :, 2] = self.vehicle_time_budget

        # reset vehicle done
        self.vehicle_done = self.nodes.new_zeros((self.minibatch, self.vehicle_count), dtype=torch.bool)
        self.done = False

        # reset cust_mask
        self.customer_mask = self.nodes[:, :, 3] > 0
        if self.init_customer_mask is not None:
            self.customer_mask = self.customer_mask | self.init_customer_mask

        # reset new customers and served customer since now to zero (all false)
        self.new_customer = True
        self.served = torch.zeros_like(self.customer_mask)

        # reset mask (minibatch*veh_count*nodes)
        self.mask = self.customer_mask[:, None, :].repeat(1, self.vehicle_count, 1)

        # reset current vehicle index, current vehicle, current vehicle mask
        self.current_vehicle_index = self.nodes.new_zeros((self.minibatch, 1), dtype=torch.int64)
        
        self.current_vehicle = self.vehicles.gather(1, 
                                                    self.current_vehicle_index[:, :, None].expand(-1, -1, self.vehicle_feature))
        self.current_vehicle_mask = self.mask.gather(1, 
                                             self.current_vehicle_index[:, :, None].expand(-1, -1, self.nodes_count))

    
    def step(self, customer_index):
        dest = self.nodes.gather(1, customer_index[:, :, None].expand(-1, -1, self.customer_feature))
        dist, dyn_cust = self._update_current_vehicles(dest, customer_index)

        #cust = (dest[:, :, 3] >= 0).float()

        self._done(customer_index)
        self._update_mask(customer_index)
        self._update_next_vehicle()

        #reward = -dist * (1 - dyn_cust*self.dynamic_reward)
        reward = self.current_vehicle[:, :, 7] - self.current_vehicle[:,:,6] + self.dynamic_reward*dyn_cust
        pending_static_customers = torch.logical_and((self.served ^ True), 
                                                     (self.nodes[:, :, 3] == 0)).float().sum(-1,keepdim=True) - 1
        
        reward -= self.pending_cost*pending_static_customers

        if self.done:

            if self.init_custoemr_mask is not None:
                self.served += self.init_customer_mask
            # penalty for pending customers
            pending_customers = torch.logical_and((self.served ^ True), 
                                                  (self.nodes[:, :, 3] >= 0)).float().sum(-1, keepdim=True) - 1

            # TODO: penalty for having unused time budget as well not serving customers
            reward -= self.dynamic_reward*pending_customers
            

        self._update_dynamic_customers()

        return reward

    def state_dict(self, dest_dict=None):
        if dest_dict is None:
            dest_dict = {'vehicles': self.vehicles,
                         'vehicle_done': self.vehicle_done,
                         'served': self.served,
                         'mask': self.mask,
                         'current_vehicle_index': self.current_vehicle_index}

        else:
            dest_dict["vehicles"].copy_(self.vehicles)
            dest_dict["vehicle_done"].copy_(self.vehicle_done)
            dest_dict["served"].copy_(self.served)
            dest_dict["mask"].copy_(self.mask)
            dest_dict["current_vehicle_index"].copy_(self.current_vehicle_index)

        return dest_dict

    def load_state_dict(self, state_dict):
        self.vehicles.copy_(state_dict["vehicles"])
        self.vehicle_done.copy_(state_dict["vehicle_done"])
        self.served.copy_(state_dict["served"])
        self.mask.copy_(state_dict["mask"])
        self.current_vehicle_index.copy_(state_dict["current_vehicle_index"])

        self.current_vehicle = self.vehicles.gather(1, 
                                                    self.current_vehicle_index[:, :, None].expand(-1, -1, self.vehicle_feature))
        self.current_vehicle_mask = self.mask.gather(1, self.current_vehicle_index[:, :, None].expand(-1, -1, self.customer_feature))




In [6]:
env = DVRPSR_Environment(data)
env.reset()

In [7]:
env.step(torch.tensor([[1],[2],[3],[4]]))

tensor([[-54.0000],
        [-46.0000],
        [-36.8000],
        [-42.8000]])

In [8]:
 # reset vehicle (minibatch*veh_count*veh_feature) distance_matrix
    
minibatch = 4
vehicle_count = 2
vehicle_feature = 8
customer_feature = 4
vehicle_budget = 400
init_cust_mask = None
nodes_count = 161
customer_size = 161

In [9]:
distance_mat = data.edges_attributes.view((minibatch, customer_size, customer_size))

In [10]:


nodes = data.nodes

vehicles = nodes.new_zeros((minibatch, vehicle_count, vehicle_feature))
vehicles[:, :, :2] = data.nodes[:, :1, :2]
vehicles[:, :, 2] = data.vehicle_time_budget


# reset vehicle done
veh_done = nodes.new_zeros((minibatch, vehicle_count), dtype=torch.bool)
done = False

# reset cust_mask
customer_mask = nodes[:, :, 3] > 0
if init_cust_mask is not None:
    customer_mask = customer_mask | init_cust_mask

# reset new customers and served customer since now to zero (all false)
new_customer = True
served = torch.zeros_like(customer_mask)

# reset mask (minibatch*veh_count*nodes)
mask = customer_mask[:, None, :].repeat(1, vehicle_count, 1)

# reset current vehicle idx, current vehicle, current vehicle mask
current_vehicle_index = nodes.new_zeros((minibatch, 1), dtype=torch.int64)
current_vehicle = vehicles.gather(1, current_vehicle_index[:, :, None].expand(-1, -1, vehicle_feature))
current_vehicle_mask = mask.gather(1, current_vehicle_index[:, :, None].expand(-1, -1, nodes_count))

In [11]:
customer_index = torch.tensor([[4],[5],[2],[3]])
dest = nodes.gather(1, customer_index[:, :, None].expand(-1, -1, customer_feature))
current_vehicle[:,:,5] = customer_index

dist = torch.zeros((minibatch, 1))
for i in range(minibatch):
    dist[i,0] = distance_mat[i][int(current_vehicle[i,:,4])][int(current_vehicle[i,:,5])]
    


In [12]:
dist

tensor([[27.7970],
        [27.5712],
        [19.1865],
        [47.4011]])

In [13]:
current_vehicle

tensor([[[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   4.0000,   0.0000,
            0.0000]],

        [[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   5.0000,   0.0000,
            0.0000]],

        [[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   2.0000,   0.0000,
            0.0000]],

        [[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   3.0000,   0.0000,
            0.0000]]])

In [14]:
dist_pair = torch.pairwise_distance(current_vehicle[:, 0, :2], dest[:, 0, :2], keepdim=True)
dist_pair


tensor([[27.7970],
        [27.5712],
        [19.1865],
        [47.4011]])

In [15]:
current_vehicle

tensor([[[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   4.0000,   0.0000,
            0.0000]],

        [[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   5.0000,   0.0000,
            0.0000]],

        [[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   2.0000,   0.0000,
            0.0000]],

        [[153.4832,  -2.7868, 600.0000,   0.0000,   0.0000,   3.0000,   0.0000,
            0.0000]]])

In [16]:
cost = torch.zeros((minibatch, customer_size,1))
for i in range(minibatch):
    for j in range(customer_size):
        dist_vehicle_customer_depot = distance_mat[i][int(current_vehicle[i, :, 4])][j] + \
                                      distance_mat[i][j][0]
        cost[i,j] = dist_vehicle_customer_depot

In [17]:
cost.size()

torch.Size([4, 161, 1])

In [18]:
nodes.size()

torch.Size([4, 161, 4])

In [19]:
cost

tensor([[[  0.0000],
         [ 40.3819],
         [ 93.9552],
         [ 83.4905],
         [ 55.5940],
         [ 46.0997],
         [ 80.9013],
         [ 77.4589],
         [ 15.0983],
         [ 39.9231],
         [ 35.4436],
         [ 13.4016],
         [ 63.6982],
         [ 61.6642],
         [ 56.2730],
         [ 91.3629],
         [ 64.5679],
         [ 55.4666],
         [ 66.4132],
         [ 65.8727],
         [ 89.3819],
         [ 48.8343],
         [107.0979],
         [ 79.3199],
         [ 90.2541],
         [ 68.0518],
         [ 66.0881],
         [ 58.2116],
         [ 28.2565],
         [ 21.5203],
         [102.1403],
         [ 43.9617],
         [ 91.7802],
         [ 64.1265],
         [ 34.7705],
         [108.5564],
         [ 44.5163],
         [ 60.5798],
         [ 79.0836],
         [ 91.8384],
         [ 91.8632],
         [ 63.7811],
         [ 68.3427],
         [ 68.0784],
         [ 19.7185],
         [ 97.9313],
         [ 17.5386],
         [ 92

In [20]:
cost += nodes[:,:,None, 2]

In [21]:
overtime_mask = current_vehicle[:, :,None,2] - cost

In [22]:
overtime_mask.size()

torch.Size([4, 161, 1])

In [23]:
overtime_mask = overtime_mask.squeeze(2).unsqueeze(1)

In [24]:
overtime_mask.size()

torch.Size([4, 1, 161])

In [25]:
overtime_mask

tensor([[[600.0000, 550.8181, 495.2048, 506.6395, 539.6160, 540.8003, 504.1887,
          514.3011, 568.9917, 551.0169, 554.6364, 575.5184, 531.4819, 528.0958,
          532.5270, 501.8971, 520.4821, 531.6234, 526.2768, 528.3973, 504.1981,
          543.3057, 482.3821, 511.6101, 496.1659, 523.9482, 519.7618, 530.6584,
          556.8535, 564.0798, 486.5397, 549.9683, 499.5698, 525.1435, 558.9095,
          483.2836, 540.5137, 528.4603, 513.1764, 496.0416, 498.9268, 525.6790,
          518.9473, 523.5016, 568.4415, 488.8987, 572.2714, 492.8947, 549.2734,
          569.2064, 528.9928, 546.7451, 500.7271, 517.1108, 530.7105, 516.4409,
          521.6797, 471.4446, 563.3617, 538.9496, 558.4188, 576.8260, 526.9244,
          573.2993, 538.7860, 506.7955, 500.2465, 535.5887, 515.2085, 532.6863,
          564.9778, 547.0604, 501.1992, 498.4866, 497.2979, 529.2914, 547.2241,
          507.8678, 555.5311, 494.2935, 509.6904, 509.4359, 508.3792, 515.2063,
          484.2237, 542.7291, 493.7278, 

In [26]:

overtime = torch.zeros_like(mask).scatter_(1,
                                    current_vehicle_index[:, :,None].expand(-1, -1, nodes_count),
                                    overtime_mask < 0)

mask = mask | served[:, None, :] | overtime | veh_done[:, :, None]
mask[:, :, 0] = 0  # depot

In [27]:
current_vehicle_index[:, :,None].expand(-1, -1, nodes_count).size()

torch.Size([4, 1, 161])

In [28]:
overtime_mask.size()

torch.Size([4, 1, 161])

In [29]:
mask.size()

torch.Size([4, 2, 161])

In [30]:
served.size()

torch.Size([4, 161])

In [31]:
overtime.size()

torch.Size([4, 2, 161])

In [32]:
veh_done.size()

torch.Size([4, 2])

In [2]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.data import Data,DataLoader
from tqdm import tqdm
def creat_instance(num,n_nodes=100,random_seed=None):
    if random_seed is None:
        random_seed = np.random.randint(123456789)
    np.random.seed(random_seed)
    def random_tsp(n_nodes,random_seed=None):

        data = np.random.uniform(0,1,(n_nodes,2))
        return data
    datas = random_tsp(n_nodes)

    def c_dist(x1,x2):
        return ((x1[0]-x2[0])**2+(x1[1]-x2[1])**2)**0.5
    #edges = torch.zeros(n_nodes,n_nodes)
    edges = np.zeros((n_nodes,n_nodes,1))

    for i, (x1, y1) in enumerate(datas):
        for j, (x2, y2) in enumerate(datas):
            d = c_dist((x1, y1), (x2, y2))
            edges[i][j][0]=d
    edges = edges.reshape(-1, 1)
    CAPACITIES = {
        10: 2.,
        20: 3.,
        50: 4.,
        160: 5.
    }

    demand = np.random.randint(1, 10, size=(n_nodes-1)) # Demand, uniform integer 1 ... 9
    demand = np.array(demand)/10
    demand = np.insert(demand,0,0.)
    capcity = CAPACITIES[n_nodes-1]
    return datas,edges,demand,capcity#demand(num,node) capcity(num)

'''a,s,d,f = creat_instance(2,21)
print(d,f)'''
def creat_data(n_nodes,num_samples=10000 ,batch_size=32):
    edges_index = []
    for i in range(n_nodes):
        for j in range(n_nodes):
            edges_index.append([i, j])
    edges_index = torch.LongTensor(edges_index)
    edges_index = edges_index.transpose(dim0=0,dim1=1)

    datas = []

    for i in range(num_samples):
        node, edge, demand, capcity = creat_instance(num_samples, n_nodes)
        data = Data(x=torch.from_numpy(node).float(), edge_index=edges_index,edge_attr=torch.from_numpy(edge).float(),
                    demand=torch.tensor(demand).unsqueeze(-1).float(),capcity=torch.tensor(capcity).unsqueeze(-1).float())
        
        datas.append(data)
    dl = DataLoader(datas, batch_size=batch_size)
    return dl

In [3]:
#dl = creat_data(161, 4, 4)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import math
from torch.distributions.categorical import Categorical
from torch.optim.lr_scheduler import LambdaLR
import time

from torch import Tensor

import torch_geometric.typing
from torch_geometric.typing import torch_scatter

#from torch_scatter import segment


In [5]:
INIT = True

class GatConv(MessagePassing):
    def __init__(self, in_channels, out_channels, edge_channels,
                 negative_slope=0.2, dropout=0):
        super(GatConv, self).__init__(aggr='add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.fc = nn.Linear(in_channels, out_channels)
        self.attn = nn.Linear(2 * out_channels + edge_channels, out_channels)
        if INIT:
            for name, p in self.named_parameters():
                if 'weight' in name:
                    if len(p.size()) >= 2:
                        nn.init.orthogonal_(p, gain=1)
                elif 'bias' in name:
                    nn.init.constant_(p, 0)

    def forward(self, x, edge_index, edge_attr, size=None):
        x = self.fc(x)
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, edge_index_i, x_i, x_j, edge_attr):
        x = torch.cat([x_i, x_j, edge_attr], dim=-1)
        alpha = self.attn(x)
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, edge_index_i)

        # Sample attention coefficients stochastically.
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)

        return x_j * alpha

    def update(self, aggr_out):
        return aggr_out


class Encoder(nn.Module):
    def __init__(self, input_node_dim, hidden_node_dim, input_edge_dim, hidden_edge_dim, conv_layers=3, n_heads=4):
        super(Encoder, self).__init__()
        self.hidden_node_dim = hidden_node_dim
        self.fc_node = nn.Linear(input_node_dim, hidden_node_dim)
        self.bn = nn.BatchNorm1d(hidden_node_dim)
        self.be = nn.BatchNorm1d(hidden_edge_dim)
        self.fc_edge = nn.Linear(input_edge_dim, hidden_edge_dim)  # 1-16
        # self.bn = nn.ModuleList([nn.BatchNorm1d(hidden_node_dim) for i in range(conv_layers)])
        # self.convs = nn.ModuleList([GatConv(hidden_node_dim, hidden_node_dim, hidden_edge_dim) for i in range(n_heads)])
        self.convs1 = nn.ModuleList(
            [GatConv(hidden_node_dim, hidden_node_dim, hidden_edge_dim) for i in range(conv_layers)])

        # self.convs = nn.ModuleList([GATConv(hidden_node_dim, hidden_node_dim) for i in range(conv_layers)])
        if INIT:
            for name, p in self.named_parameters():
                if 'weight' in name:
                    if len(p.size()) >= 2:
                        nn.init.orthogonal_(p, gain=1)
                elif 'bias' in name:
                    nn.init.constant_(p, 0)

    def forward(self, data):
        batch_size = data.num_graphs
        # print(batch_size)
        # edge_attr = data.edge_attr

        x = torch.cat([data.x, data.demand], -1)
        x = self.fc_node(x)
        x = self.bn(x)
        edge_attr = self.fc_edge(data.edge_attr)
        edge_attr = self.be(edge_attr)
        for conv in self.convs1:
            # x = conv(x,data.edge_index)
            x1 = conv(x, data.edge_index, edge_attr)
            x = x + x1

        x = x.reshape((batch_size, -1, self.hidden_node_dim))

        return x


In [15]:
dl = creat_data(161, 4, 4)

In [16]:
encoder = Encoder(input_node_dim = 3, hidden_node_dim=128, input_edge_dim=1, hidden_edge_dim=16, conv_layers=3)
encoder.to(device)

Encoder(
  (fc_node): Linear(in_features=3, out_features=128, bias=True)
  (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (be): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_edge): Linear(in_features=1, out_features=16, bias=True)
  (convs1): ModuleList(
    (0-2): 3 x GatConv(128, 128)
  )
)

In [17]:
n_nodes = 161
batch_size = 4
device = torch.device('cpu')

for batch_idx, batch in enumerate(dl):
    
    x,attr,capcity,demand = batch.x,batch.edge_attr,batch.capcity,batch.demand
    print(x.size(),attr.size())
    
    x,attr,capcity,demand = x.view((batch_size,n_nodes,2)), attr.view(batch_size,n_nodes*n_nodes,1),capcity.view(batch_size,1), demand.view(batch_size, n_nodes,1)
    print(x.size(),attr.size())
    
    batch = batch.to(device)
    
    encode = encoder(batch)
    
    print(batch.edge_index.size(), batch.x.size())

torch.Size([644, 2]) torch.Size([103684, 1])
torch.Size([4, 161, 2]) torch.Size([4, 25921, 1])
torch.Size([2, 103684]) torch.Size([644, 2])
